In [23]:
import numpy as np
from ppopt.mplp_program import MPLP_Program
from ppopt.mpmodel import MPModeler
from ppopt.mp_solvers.solve_mpqp import solve_mpqp, mpqp_algorithm
from ppopt.plot import parametric_plot
from pyomo.environ import *
import pyomo.environ as pyo
# from numpy.polynomial.legendre import leggauss
# from scipy.optimize import linprog
# from collections import defaultdict
# import time
# import sympy as sp
# from sympy.logic.boolalg import BooleanTrue, BooleanFalse
from typing import List, Tuple
# import pickle
# import math
# import itertools
# import numpy as np
# from IPython.display import display

In [24]:
def get_sources(trans_list:List[Tuple], sinks:list):
    sources = {i: None for i in sinks}
    for sink in sinks:
        so_list = list()
        for pair in trans_list:
            if pair[1]==sink:
                so_list.append(pair[0])
        sources[sink] = so_list
    return sources

In [25]:
def get_sinks(trans_list:List[Tuple], sources:list):
    sinks = {i: None for i in sources}
    for source in sources:
        si_list = list()
        for pair in trans_list:
            if pair[0] == source:
                si_list.append(pair[1])
        sinks[source] = si_list
    return sinks

In [26]:
loc_indexes = [i for i in range(1,6)]
loc_capacities = [60, 70, 120, 40, 150]
markets = [5]
loc_for_design = [3]
cap_capex_dict = {3:50}
cap_opex_dict = {3:2}

transports = [(1,3), (2, 3), (2, 4), (3, 5), (4, 3), (4, 5)]
trans_costs = [0.24, 0.37, 0.41, 0.29, 0.46, 0.33]
source_dict = get_sources(trans_list=transports, sinks=loc_indexes)
sink_dict = get_sinks(trans_list=transports, sources=loc_indexes)
loc_inters = [i for i in loc_indexes if (source_dict[i] and sink_dict[i])]

demand_bounds = {5: (70, 130)}
capf_bounds = {4:(0, 0.5)}
nt = 2

loc_capacities_dict = {j: loc_capacities[i] for i,j in enumerate(loc_indexes)}
trans_costs_dict = {j: trans_costs[i] for i,j in enumerate(transports)}

## FLEXIBILITY TEST

In [27]:
model = MPModeler()

u = model.add_var('u')
f = {(i[0],i[1]): model.add_var(name=f'f[{i[0]},{i[1]}]') for i in transports}

d = {m: model.add_param(name=f'd_{m}') for m in markets}
capf = {i: model.add_param(name=f'capf_{i}') for i in capf_bounds}

model.add_constrs([(sum(f[so, si] for si in sink_dict[so]) if sink_dict[so] else sum(f[sp, so] for sp in source_dict[so])) <= (
        capf[so] * loc_capacities_dict[so] + u if so in capf_bounds else loc_capacities_dict[so] + u) for so in loc_indexes if so not in loc_for_design])

if loc_for_design:
    loc_cap = {i: model.add_param(name=f'cap_{i}') for i in loc_for_design}
    y_cap = {i: model.add_var(name=f'y_{i}') for i in loc_for_design}
    model.add_constrs([(sum(f[so, si] for si in sink_dict[so]) if sink_dict[so] else sum(f[sp, so] for sp in source_dict[so])) <=
                       (loc_cap[so] + u) for so in loc_for_design])
    model.add_constrs([loc_cap[so] <= y_cap[so]*loc_capacities_dict[so] + u for so in loc_for_design])

model.add_constrs([sum(f[i, si] for si in sink_dict[i]) == sum(f[so, i] for so in source_dict[i]) for i in loc_inters])
model.add_constrs([sum(f[so,m] for so in source_dict[m]) + u >= d[m] for m in markets])

model.add_constrs([f[i]>=0 for i in transports])

model.add_constrs(d[m] <= demand_bounds[m][1] for m in markets)
model.add_constrs(d[m] >= demand_bounds[m][0] for m in markets)
model.add_constrs(capf[l] <= capf_bounds[l][1] for l in capf_bounds)
model.add_constrs(capf[l] >= capf_bounds[l][0] for l in capf_bounds)

model.set_objective(u)
prog = model.formulate_problem()
prog.process_constraints()

## COST MINIMIZATION - FIXED DESIGN

In [28]:
# model = MPModeler()
# 
# # u = model.add_var('u')
# f = {(i[0],i[1]): model.add_var(name=f'f[{i[0]},{i[1]}]') for i in transports}
# 
# d = {m: model.add_param(name=f'd_{m}') for m in markets}
# capf = {i: model.add_param(name=f'capf_{i}') for i in capf_bounds}
# 
# model.add_constrs([(sum(f[so, si] for si in sink_dict[so]) if sink_dict[so] else sum(f[sp, so] for sp in source_dict[so])) <= (
#         capf[so] * loc_capacities_dict[so] if so in capf_bounds else loc_capacities_dict[so]) for so in loc_indexes])
# 
# model.add_constrs([sum(f[i, si] for si in sink_dict[i]) == sum(f[so, i] for so in source_dict[i]) for i in loc_inters])
# model.add_constrs([sum(f[so,m] for so in source_dict[m]) >= d[m] for m in markets])
# 
# model.add_constrs([f[i]>=0 for i in transports])
# 
# model.add_constrs(d[m] <= demand_bounds[m][1] for m in markets)
# model.add_constrs(d[m] >= demand_bounds[m][0] for m in markets)
# model.add_constrs(capf[l] <= capf_bounds[l][1] for l in capf_bounds)
# model.add_constrs(capf[l] >= capf_bounds[l][0] for l in capf_bounds)
# 
# obj_expr = sum(f[j]*trans_costs_dict[j] for j in transports)
# 
# model.set_objective(obj_expr)
# prog = model.formulate_problem()
# prog.process_constraints()

## COST MINIMIZATION WITH DESIGN OPTIMIZATION

In [29]:
# model = MPModeler()
# 
# # u = model.add_var('u')
# f = {(i[0],i[1]): model.add_var(name=f'f[{i[0]},{i[1]}]') for i in transports}
# 
# d = {m: model.add_param(name=f'd_{m}') for m in markets}
# capf = {i: model.add_param(name=f'capf_{i}') for i in capf_bounds}
# 
# model.add_constrs([(sum(f[so, si] for si in sink_dict[so]) if sink_dict[so] else sum(f[sp, so] for sp in source_dict[so])) <= (
#         capf[so] * loc_capacities_dict[so] if so in capf_bounds else loc_capacities_dict[so]) for so in loc_indexes if so not in loc_for_design])
# 
# if loc_for_design:
#     loc_cap = {i: model.add_param(name=f'cap_{i}') for i in loc_for_design}
#     y_cap = {i: model.add_var(name=f'y_{i}') for i in loc_for_design}
#     model.add_constrs([(sum(f[so, si] for si in sink_dict[so]) if sink_dict[so] else sum(f[sp, so] for sp in source_dict[so])) <=
#                        (loc_cap[so]) for so in loc_for_design])
#     model.add_constrs([loc_cap[so] <= y_cap[so]*loc_capacities_dict[so] for so in loc_for_design])
# 
# model.add_constrs([sum(f[i, si] for si in sink_dict[i]) == sum(f[so, i] for so in source_dict[i]) for i in loc_inters])
# model.add_constrs([sum(f[so,m] for so in source_dict[m]) >= d[m] for m in markets])
# 
# model.add_constrs([f[i]>=0 for i in transports])
# 
# model.add_constrs(d[m] <= demand_bounds[m][1] for m in markets)
# model.add_constrs(d[m] >= demand_bounds[m][0] for m in markets)
# model.add_constrs(capf[l] <= capf_bounds[l][1] for l in capf_bounds)
# model.add_constrs(capf[l] >= capf_bounds[l][0] for l in capf_bounds)
# 
# obj_expr = sum(f[j]*trans_costs_dict[j] for j in transports)
# if loc_for_design:
#     obj_expr += sum(y_cap[i]*cap_capex_dict[i] + loc_cap[i]*cap_opex_dict[i] for i in loc_for_design)
# 
# model.set_objective(obj_expr)
# prog = model.formulate_problem()
# prog.process_constraints()

In [30]:
solution = solve_mpqp(prog, mpqp_algorithm.combinatorial)

In [31]:
# solution

In [32]:
parametric_plot(solution, show=True)

Solution is not 2D, the dimensionality of the solution is 3


In [33]:
len(solution)

44

In [34]:
psi_list = list()
for cr in solution.critical_regions:
    psi_list.append(np.concatenate([cr.A, cr.b], axis=1)[0])

In [35]:
len(psi_list)

44

In [36]:
psi_list

[array([ 3.33333333e-01, -2.40101513e-15,  6.43476554e-17, -4.33333333e+01]),
 array([ 3.33333333e-01, -9.18908557e-16, -6.51846173e-19, -4.33333333e+01]),
 array([ 3.33333333e-01, -6.99480447e-16,  1.08169905e-18, -4.33333333e+01]),
 array([ 0.00000000e+00, -4.00000000e+01,  2.84364115e-17, -1.32453639e-13]),
 array([ 0.00000000e+00, -4.00000000e+01, -4.52893403e-17,  2.40343420e-13]),
 array([ 0.00000000e+00, -4.00000000e+01,  3.67070131e-19,  2.05690997e-13]),
 array([ 3.33333333e-01,  0.00000000e+00, -3.72549485e-16, -4.33333333e+01]),
 array([ 3.33333333e-01,  0.00000000e+00, -3.23246716e-17, -4.33333333e+01]),
 array([ 3.33333333e-01,  0.00000000e+00, -9.63120979e-19, -4.33333333e+01]),
 array([ 3.33333333e-01,  0.00000000e+00, -1.61000164e-20, -4.33333333e+01]),
 array([ 3.33333333e-01, -1.33333333e+01, -3.33333333e-01,  7.28464816e-14]),
 array([ 0.00000000e+00, -4.00000000e+01, -2.11350519e-16,  8.54447544e-15]),
 array([ 0.00000000e+00, -4.00000000e+01,  3.26029304e-16, -5.20